## 准备数据

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=(
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

docs = text_splitter.split_documents(documents)

docs[1]

USER_AGENT environment variable not set, consider setting it to identify your requests.


Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a 

## 实例化Milvus的向量数据库，该数据库会将文档加载到Milvs向量数据库并在后台构建索引

In [2]:
import sys
from langchain_milvus import Milvus
sys.path.append("..")
from llm_utils import qwen_embeddings,llm,openai_embedding

docs = docs[:10]  # 限制文档数量，避免批量过大报错
vectorstore = Milvus.from_documents(
    documents=docs,
    # embedding=openai_embedding,
    embedding=qwen_embeddings,
    connection_args={
        "uri": "http://localhost:19530",  # http://localhost:19530
    },
    collection_name="langchain_example",
    drop_old=True,  # 删除现有集合以解决维度不匹配问题
)

2025-08-30 22:16:07,729 [DEBUG][_create_connection]: Created new connection using: async-http://localhost:19530 (async_milvus_client.py:599)


## 检索

In [3]:
query = "What is self-reflection of an AI Agent?"
vectorstore.similarity_search(query, k=1)     # 实际上后续用LCEL 实现 用构建好的 retrieval |  format_docs 来拿到模型期望的<context>字符串格式类型

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'pk': 460472533254406470}, page_content='Examples of reasoning trajectories for knowledge-intensive tasks (e.g. HotpotQA, FEVER) and decision-making tasks (e.g. AlfWorld Env, WebShop). (Image source: Yao et al. 2023).\n\nIn both experiments on knowledge-intensive tasks and decision-making tasks, ReAct works better than the Act-only baseline where Thought: … step is removed.\nReflexion (Shinn & Labash 2023) is a framework to equip agents with dynamic memory and self-reflection capabilities to improve reasoning skills. Reflexion has a standard RL setup, in which the reward model provides a simple binary reward and the action space follows the setup in ReAct where the task-specific action space is augmented with language to enable complex reasoning steps. After each action $a_t$, the agent computes a heuristic $h_t$ and optionally may decide to reset the environment to start a new trial depending on the 

In [4]:
# Define the prompt template for generating AI responses
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# 您是一个人工智能助手，在可能的情况下会基于事实和统计数据来回答问题。
# 请利用以下信息为<question>标签内的问题提供简洁回答。
# 若不知道答案，请直接说明不清楚，不要试图编造答案。
PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

prompt = PromptTemplate(
    template = PROMPT_TEMPLATE,
    input_variables = ["context", "question"]
)

# 构建检索器retrieval，便于集成到LCEL
retrieval = vectorstore.as_retriever()

# 定于检索器输出处理函数
def format_docs(docs):
    """Format the retrieved documents into a single string."""
    return "\n\n".join(doc.page_content for doc in docs)

## 构建LECL处理逻辑链

In [5]:
rag_chain = (
    {"context": retrieval |  format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# rag_chain.get_graph().print_ascii()

# Invoke the RAG chain with a specific question and retrieve the response
# res = rag_chain.invoke(query)
# print(res)
# stream输出
for chunk in rag_chain.stream(query):
    print(chunk, end="|", flush=True) 

|Self|-ref|lection| of| an| AI| agent| is| a| crucial| process| that| allows| the| agent| to| improve| iter|atively| by| analyzing| and| refining| its| past| actions|.| It| involves| evaluating| previous| decision| trajectories| to| learn| from| mistakes| and| correct| errors| for| future| tasks|.| This| is| typically| achieved| through| mechanisms| such| as| storing| and| utilizing| past| reflections|,| guided| by| examples| of| failed| trajectories| and| ideal| responses|,| which| can| enhance| the| agent|'s| reasoning| and| decision|-making| skills|.| In| experiments|,| self|-ref|lection| has| been| shown| to| be| vital|,| particularly| in| complex| tasks|,| and| is| part| of| frameworks| like| Reflex|ion|,| which| integrates| dynamic| memory| and| self|-e|valuation| to| improve| reasoning| abilities|.||